In [ ]:
from music21 import midi
import pretty_midi
import torch
import numpy as np
from copy import copy, deepcopy
import pysnooper
import time

In [1]:
def numpy_to_midi(sample_roll, output_folder='demo', listen = False, save = True):
    music = pretty_midi.PrettyMIDI()
    piano_program = pretty_midi.instrument_name_to_program(
        'Acoustic Grand Piano')
    piano = pretty_midi.Instrument(program=piano_program)
    t = 0
    for i in sample_roll:
        if 'torch' in str(type(i)):
            pitch = int(i.max(0)[1])
        else:
            pitch = int(np.argmax(i))
        if pitch < 128:
            note = pretty_midi.Note(
                velocity=100, pitch=pitch, start=t, end=t + 1 / 8)
            t += 1 / 8
            piano.notes.append(note)
        elif pitch == 128:
            if len(piano.notes) > 0:
                note = piano.notes.pop()
            else:
                p = np.random.randint(60, 72)
                note = pretty_midi.Note(
                    velocity=100, pitch=int(p), start=0, end=t)
            note = pretty_midi.Note(
                velocity=100,
                pitch=note.pitch,
                start=note.start,
                end=note.end + 1 / 8)
            piano.notes.append(note)
            t += 1 / 8
        elif pitch == 129:
            t += 1 / 8
    music.instruments.append(piano)
    if output_folder is None:
        output = str(time.time()) + '.mid'
    else:
        output = output_folder + '/' + str(time.time()) + '.mid'
    if save:
        music.write(output)
    if listen:
        midi_listener(output)

SyntaxError: invalid syntax (<ipython-input-1-9fa7f2ea4068>, line 35)

In [2]:
def midi_listener(midi_file_path):
    mf = midi.MidiFile()
    mf.open(midi_file_path)
    mf.read()
    mf.close()
    s = midi.translate.midiFileToStream(mf)
    s.show('midi')

In [ ]:
def data_augment(musicdata_list, data_length):
    output_list = list()
    aug_time_stats = list()
    
    for i, data_object in enumerate(musicdata_list):
        aug_time = len(data_object.torch_matrix) // data_length
        # print(len(data_object.torch_matrix), data_object.torch_matrix.shape)
        aug_time_stats.append(aug_time)
        if aug_time >= 1:
            for j in range(aug_time):
                temp_data = copy(data_object)
                temp_data.torch_matrix = data_object.torch_matrix[data_length*j:data_length*(j+1)]
                output_list.append(temp_data)
    # print(aug_time_stats)
    return output_list